# **Resolving Partially Ordered Traces Using Deep Learning** (Seq2Seq)


|                        | BPI 2012| BPI 2014 | Traffic |
|-----------------------:|--------:|---------:|--------:|
| \|A\|                  | 24      |  9       | 11      |
| #Traces                | 13087   | 41353    | 150370  |
| #Events                | 262200  | 369485   | 561470  |
| #Event Sets            | 248205  | 243186   | 549452  |
| #uncertain Seq's       | 14      | 24       | 25      |
| Trace Uncertainty      | 38%     | 93%      |  6%     |
| Event Uncertainty      |  5%     | 40%      |  2%     |
| max(len(unc.seq))      |  4      |  4       |  3      |
| avg(len(unc.seq))      |  2.4    |  2.6     |  2.0    |

### imports and PIP installs

In [ ]:
!pip install pm4py

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import sys
sys.path.append('/content/drive/MyDrive/Bachelor_Thesis/')

In [4]:
import utils

In [5]:
from tqdm import tqdm
import itertools
from itertools import combinations_with_replacement, product
from random import shuffle
from pm4py.objects.log.importer.xes import importer as xes_importer

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from sklearn import model_selection
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, Input, LSTM

### Loading the logs

In [55]:
#b12_log = xes_importer.apply("/content/drive/MyDrive/Bachelor_Thesis/logs/BPI_Challenge_2012.xes")
#b14_log = xes_importer.apply("/content/drive/MyDrive/Bachelor_Thesis/logs/BPI_Challenge_2014.xes")
traffic_log = xes_importer.apply("/content/drive/MyDrive/Bachelor_Thesis/logs/traffic_fines.xes")

In [ ]:
# artificial logs

#a_log0   = xes_importer.apply("./logs/generated_logs/1561989897361-4_0.xes")
#a_log25  = xes_importer.apply("./logs/generated_logs/1561989897361-4_25.xes")
#a_log50  = xes_importer.apply("./logs/generated_logs/1561989897361-4_50.xes")
#a_log75  = xes_importer.apply('/content/drive/MyDrive/Bachelor_Thesis/logs/generated_logs/1561989897313-3_75.xes')
#a_log100 = xes_importer.apply("./logs/generated_logs/1561989906794-495_100.xes")

# **Seq2Seq**
---

In [56]:
log = utils.remove_timezones(traffic_log)
#utils.abstract_time(log, utils.abstract_seconds) # AF abstract to minutes for the artificial logs
c_log, u_log = utils.split_log(log)

sparse_log = utils.get_sparse_log(log)
sparse_c_log, sparse_u_log = utils.get_sparse_log(c_log), utils.get_sparse_log(u_log)

A = list(set([event["concept:name"] for trace in log for event in trace ]))
A_set = [[activity] for activity in A]

#log_set   = utils.get_sparse_log_set(log)      
#c_log_set = utils.get_sparse_log_set(c_log)
#u_log_set = utils.get_sparse_log_set(u_log)

log_set   = utils.get_sparse_log_set_artificial(log)     # AF build log sets for the artificial logs, i.e. only direct neighbor with equal timestamps are equal 
c_log_set = utils.get_sparse_log_set_artificial(c_log)
u_log_set = utils.get_sparse_log_set_artificial(u_log)

In [57]:
max_trace_len = utils.longest_trace(u_log)
max_unc_trace_len = utils.longest_trace(u_log_set)
max_seq_len = utils.longest_unc_seq(u_log_set)
k = max_seq_len # longest uncertain sequences
print(k)

3


In [58]:
unc_seq = utils.possible_uncertain_seq(A, k) 
pos_res = utils.possible_resolutions(A, k)

In [59]:
NAME = "concept:name"
TIME = "time:timestamp"
for event in log[555]: 
  print("{:20} at   {}".format(event[NAME], event[TIME]))

Create Fine          at   2007-04-16 02:00:00
Payment              at   2007-04-16 02:00:00


In [60]:
pos_res_for_unc_seq = utils.pos_res_for_unc_seq(unc_seq)

In [61]:
# add start and end sequence symbol to each target trace
BOS = '<'
EOS = '>'

In [62]:
# shrink the set if it is larger 10000 to 10000 
# since the smallest set has about 10000 (traffic log) 
# also for huge encoding 2, 3
c = list(zip(u_log_set, sparse_u_log))
shuffle(c)
u_log_set, sparse_u_log = zip(*c)
u_log_set, sparse_u_log = list(u_log_set), list(sparse_u_log)

enc1 = True

if len(u_log_set) > 10000:
    u_log_set = u_log_set[:10000]         # this is dec_X
    sparse_u_log = sparse_u_log[:10000]   # this is dec_y

### Encoding 1

In [63]:
rev_X = [trace[::-1] for trace in u_log_set] 
y = [[BOS] + seq + [EOS] for seq in sparse_u_log]

In [64]:
input_events = A
target_events = sorted(A + [BOS, EOS])
n_enc_tokens = len(input_events)
n_dec_tokens = len(target_events)
max_enc_seq_len = max([len(trace) for trace in rev_X])
max_dec_seq_len = max([len(trace) for trace in y])

print('Number of samples:', len(rev_X))
print('Number of unique input tokens:', n_enc_tokens)
print('Number of unique output tokens:', n_dec_tokens)
print('Max sequence length for inputs:', max_enc_seq_len)
print('Max sequence length for outputs:', max_dec_seq_len)

Number of samples: 9166
Number of unique input tokens: 11
Number of unique output tokens: 13
Max sequence length for inputs: 18
Max sequence length for outputs: 22


In [65]:
# lookup tables
INactivity_to_idx = dict( (tuple(e_set),i) for i,e_set in enumerate(input_events)) 
INidx_to_activity = dict( (i,tuple(e_set)) for i,e_set in enumerate(input_events))
INactivity_to_idx2 = dict( (e_set,i) for i,e_set in enumerate(input_events)) # for decoding
INidx_to_activity2 = dict( (i,e_set) for i,e_set in enumerate(input_events)) # for decoding

OUTactivity_to_idx = dict( (tuple(e_set),i) for i,e_set in enumerate(target_events))
OUTidx_to_activity = dict( (i,tuple(e_set)) for i,e_set in enumerate(target_events))
OUTactivity_to_idx2 = dict( (e_set,i) for i,e_set in enumerate(target_events)) # for decoding
OUTidx_to_activity2 = dict( (i,e_set) for i,e_set in enumerate(target_events)) # for decoding

In [66]:
encoder_input_data, decoder_input_data, decoder_target_data = utils.seq2seq_encode(rev_X, y, max_enc_seq_len, n_enc_tokens,
                                                                            max_dec_seq_len, n_dec_tokens, INactivity_to_idx,
                                                                            OUTactivity_to_idx, 1)

### Encoding 2 & 3

In [ ]:
rev_X = [trace_set[::-1] for trace_set in u_log_set]
y = [[[BOS]] + seq + [[EOS]] for seq in u_log_set]

In [ ]:
input_events = unc_seq
target_events = sorted(pos_res + [[BOS], [EOS]])
n_enc_tokens = len(input_events)
n_dec_tokens = len(target_events)
max_enc_seq_len = max([len(trace) for trace in rev_X]) # fix here
max_dec_seq_len = max([len(trace) for trace in y])

print('Number of samples:', len(rev_X))
print('Number of unique input tokens:', n_enc_tokens)
print('Number of unique output tokens:', n_dec_tokens)
print('Max sequence length for inputs:', max_enc_seq_len)
print('Max sequence length for outputs:', max_dec_seq_len)

Number of samples: 10000
Number of unique input tokens: 714
Number of unique output tokens: 7382
Max sequence length for inputs: 91
Max sequence length for outputs: 93


In [ ]:
# lookup tables
unc_seq_to_idx = dict( (tuple(e_set),i) for i,e_set in enumerate(input_events)) 
idx_to_unc_seq = dict( (i,tuple(e_set)) for i,e_set in enumerate(input_events))

pos_res_to_idx = dict( (tuple(e_set),i) for i,e_set in enumerate(target_events))
idx_to_pos_res = dict( (i,tuple(e_set)) for i,e_set in enumerate(target_events))

Encoding 2

In [ ]:
encoder_input_data, decoder_input_data, decoder_target_data = utils.seq2seq_encode(rev_X, y, max_enc_seq_len, n_enc_tokens,
                                                                            max_dec_seq_len, n_dec_tokens, unc_seq_to_idx,
                                                                            pos_res_to_idx, 2)

Encoding 3

In [ ]:
encoder_input_data, decoder_input_data, decoder_target_data = utils.seq2seq_encode(rev_X, y, max_enc_seq_len, n_enc_tokens,
                                                                            max_dec_seq_len, n_dec_tokens, unc_seq_to_idx,
                                                                            pos_res_to_idx, 3)

### split inot train and test set (test size 20%)

In [67]:
n_samples = len(rev_X)
cut = int(n_samples*0.8)
#training
train_encoder_input_data  = encoder_input_data[:cut]
train_decoder_input_data  = decoder_input_data[:cut]
train_decoder_target_data = decoder_target_data[:cut]

# test
test_encoder_input_data = encoder_input_data[cut:]
test_decoder_input_data = decoder_input_data[cut:]
test_decoder_target_data = decoder_target_data[cut:]

In [68]:
train_encoder_input_data.shape, test_encoder_input_data.shape

((7332, 18, 11), (1834, 18, 11))

## Model (Preprocessing, Training, Predictions, Evaluation)

In [69]:
batch_size = 64  # batch size for training
epochs = 30 # 50  # number of epochs to train for, 100
latent_dim = 256  # latent dimensionality of the encoding space
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3) # early stopping when no improvement on loss in 3 consequtive epochs

In [70]:
encoder_inputs = Input(shape=(None, n_enc_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, n_dec_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(n_dec_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model(inputs=[encoder_inputs, decoder_inputs], 
              outputs=decoder_outputs)

In [71]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
#model.summary()

In [72]:
history = model.fit([train_encoder_input_data, train_decoder_input_data], train_decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2,
          callbacks=[callback])

Epoch 1/30
92/92 [==============================] - 25s 210ms/step - loss: 0.2645 - val_loss: 0.2078
Epoch 2/30
92/92 [==============================] - 18s 196ms/step - loss: 0.1761 - val_loss: 0.1472
Epoch 3/30
92/92 [==============================] - 18s 200ms/step - loss: 0.1473 - val_loss: 0.1328
Epoch 4/30
92/92 [==============================] - 18s 201ms/step - loss: 0.1272 - val_loss: 0.1108
Epoch 5/30
92/92 [==============================] - 18s 199ms/step - loss: 0.1094 - val_loss: 0.0963
Epoch 6/30
92/92 [==============================] - 18s 194ms/step - loss: 0.0947 - val_loss: 0.0897
Epoch 7/30
92/92 [==============================] - 19s 208ms/step - loss: 0.0902 - val_loss: 0.0818
Epoch 8/30
92/92 [==============================] - 19s 208ms/step - loss: 0.0855 - val_loss: 0.0824
Epoch 9/30
92/92 [==============================] - 18s 198ms/step - loss: 0.0830 - val_loss: 0.0805
Epoch 10/30
92/92 [==============================] - 18s 199ms/step - loss: 0.0808 - val_lo

In [ ]:
model.save('./outputs/seq2seqsets_bpi14log_logorder_01022021.h5') 

### Make Predictions

In [73]:
#use the trained model to make predictions via inferencing
#for that we take the uncertain log as input and assume the order in the log as the correct order

#inference mode brakedown
# 1 encode input sequence and return corresponding internal states
# 2 start decoder with BOS symbol and the encoders internal states as input
# 3 append predicted activity (after looking up in lookup table) to the predicted sequence
# 4 repeat process with the previously predicted activity and the updated internal states as input
# 5 end when EOS was predicted

encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model([decoder_inputs] + decoder_states_inputs,
                      [decoder_outputs] + decoder_states)

### Evaluation of Encoding 2 and 3

In [ ]:
# build function to decode predictions 
def decode_seq(enc_input_seq, dec_input_seq, OUTact_to_idx, OUTidx_to_act):
    # encode the input sequence to get the internal state vectors
    states_value = encoder_model.predict(enc_input_seq)

    # generate empty target sequence of len 1 with only the start character
    target_seq = np.zeros((1,1, n_dec_tokens))
    target_seq[0, 0, OUTact_to_idx[tuple([BOS])]] = 1.0

    #output seq loop
    stop_cond = False
    decoded_trace = []
    num_dec_events = 0
    while not stop_cond:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        #sample token and add corresponding activity to the decoded trace
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_activity = list(OUTidx_to_act[sampled_token_index]) 
        
        #print(sampled_activity)
        #print(pos_res_for_unc_seq[tuple(dec_input_seq[num_dec_events])])
        
        # check if the sampled activity is actually a possible resolution fo that case
        while not sampled_activity in pos_res_for_unc_seq[tuple(dec_input_seq[num_dec_events])]:
            # if not take the prediction with the 2nd highest prob... etc.
            output_tokens[0, -1, sampled_token_index] = 0.0 # set the old idx with max prob to zero
            sampled_token_index = np.argmax(output_tokens[0, -1, :])
            sampled_activity = list(OUTidx_to_act[sampled_token_index])
            
            # check if all idx that had a prob > 0 where turned to 0
            # i.e. all of the possible resolution were not considered likely at all
            if np.all(output_tokens[0, -1]):
                print('BREAK: no prob...')
                sampled_activity = ['BREAK']
                break
        
        decoded_trace.append(sampled_activity)
        num_dec_events += 1

        #check for stop condition: either hitting max length or prediciting EOS
        if (sampled_activity == tuple([EOS]) or len(decoded_trace) > max_dec_seq_len or
            num_dec_events >= len(dec_input_seq) ):
            stop_cond = True

        #update the target sequence (len 1)
        target_seq = np.zeros((1, 1, n_dec_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        #update states
        states_value = [h, c]

    return decoded_trace 

In [ ]:
decoded_X_test = utils.decode_X(test_encoder_input_data, idx_to_unc_seq, mode="enc2")

In [ ]:
for idx in range(0,20):
    enc_input_seq = test_encoder_input_data[idx:idx+1]
    decoded_trace = [list(trace) for trace in decode_seq(enc_input_seq, decoded_X_test[idx][::-1],
                                                         pos_res_to_idx,
                                                         idx_to_pos_res)]
    print('-'*20)
    print('Input trace  :', decoded_X_test[idx][::-1]) # log_set[idx]
    print('Decoded trace:', decoded_trace[:])
    print(decoded_X_test[idx][::-1]==decoded_trace[:])
    print()

In [ ]:
total = len(decoded_X_test)
count = 0
for idx in tqdm(range(total)):
    enc_input_seq = test_encoder_input_data[idx:idx+1]
    decoded_trace = [list(trace) for trace in decode_seq(enc_input_seq, 
                                                         decoded_X_test[idx][::-1],
                                                         pos_res_to_idx,
                                                         idx_to_pos_res)]
    if decoded_X_test[idx][::-1] == decoded_trace[:]:
        count += 1
    #print(decoded_X[idx][::-1]==decoded_trace[:])
print(count / total)

100%|██████████| 2000/2000 [10:40<00:00,  3.12it/s]

0.177


### Special Case for ambiguous Encoding 1 

In [74]:
dec_X_test = u_log_set[cut:]
dec_y_test = sparse_u_log[cut:]

taking advantage of information about the prediction to make (length, uncertain sets size, ...)

In [ ]:
# -> instead of decoding we stay in order and use the unencoded inputs 
for idx in range(0,5):
    enc_input_seq = test_encoder_input_data[idx:idx+1]
    decoded_trace = decode_seq12(enc_input_seq, dec_X_test[idx],
                                           OUTactivity_to_idx2,
                                           OUTidx_to_activity2)
    print('-'*20)
    print("Source   :" ,dec_X_test[idx])
    print("Predicted:",decoded_trace)
    print("Target   :", dec_y_test[idx])
    print()

In [76]:
total = test_encoder_input_data.shape[0]
n_event_sets = sum([1 for trace in dec_X_test for event_set in trace])
count = 0
count_highest_prob_is_non_pos_res = 0
prediction_probabilities = {}
actual_resolution_probabilities = {}

for idx in range(total):
  enc_input_seq = test_encoder_input_data[idx:idx+1]
  decoded_trace = decode_seq12(enc_input_seq, dec_X_test[idx],
                               OUTactivity_to_idx2, OUTidx_to_activity2,
                               count_highest_prob_is_non_pos_res,
                               prediction_probabilities,
                               actual_resolution_probabilities)
  if dec_y_test[idx]  == decoded_trace:
    count += 1
  #else:
    #print("MISPREDICTED")
    #print("Source   :" ,dec_X_test[idx])
    #print("Predicted:",decoded_trace)
    #print("Target   :", dec_y_test[idx])
    #print()
  if idx%100 == 0:
    print("Source   :" ,dec_X_test[idx])
    print("Predicted:",decoded_trace)
    print("Target   :", dec_y_test[idx])
    print()
print(count / total, count_highest_prob_is_non_pos_res / n_event_sets)

Source   : [['Create Fine'], ['Send Fine'], ['Insert Fine Notification'], ['Add penalty', 'Payment']]
Predicted: ['Create Fine', 'Send Fine', 'Insert Fine Notification', 'Add penalty', 'Payment']
Target   : ['Create Fine', 'Send Fine', 'Insert Fine Notification', 'Add penalty', 'Payment']

Source   : [['Create Fine', 'Payment']]
Predicted: ['Create Fine', 'Payment']
Target   : ['Create Fine', 'Payment']

Source   : [['Create Fine', 'Payment']]
Predicted: ['Create Fine', 'Payment']
Target   : ['Create Fine', 'Payment']

Source   : [['Create Fine'], ['Send Fine'], ['Insert Fine Notification'], ['Add penalty'], ['Insert Date Appeal to Prefecture', 'Send Appeal to Prefecture']]
Predicted: ['Create Fine', 'Send Fine', 'Insert Fine Notification', 'Add penalty', 'Insert Date Appeal to Prefecture', 'Send Appeal to Prefecture']
Target   : ['Create Fine', 'Send Fine', 'Insert Fine Notification', 'Add penalty', 'Insert Date Appeal to Prefecture', 'Send Appeal to Prefecture']

Source   : [['Create

In [75]:
# build funtion to decode predictions 
def decode_seq12(enc_input_seq, dec_input_seq, OUTact_to_idx, OUTidx_to_act, count_highest_prob_is_non_pos_res,
                 prediction_probabilities, actual_resolution_probabilities):
    # encode the input sequence to get the internal state vectors
    states_value = encoder_model.predict(enc_input_seq)

    # generate empty target sequence of len 1 with only the start character
    target_seq = np.zeros((1,1, n_dec_tokens))
    target_seq[0, 0, OUTact_to_idx[BOS]] = 1.0

    #output seq loop
    stop_cond = False
    decoded_trace = []

    e_set_len = [len(event_set) for event_set in dec_input_seq]       # length of each uncertain set
    indices = [0 for length in e_set_len]                             # start indices of the uncertain set in the y_trace e.g. [[A,B], [C], [D,E]]
                                                                      #                                                   with [A, B, C, D, E] -> [0, 2, 3]
    for i in range(len(e_set_len[:-1])):
      indices[i+1] = indices[i] + e_set_len[i]


    for i in range(len(dec_input_seq)): # iterate over all event sets
          
      events_to_predict = dec_input_seq[i].copy() # the events to be predicted
          
      for j in range(e_set_len[i]): 
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # sample token and add corresponding activity to the decoded trace
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_activity = OUTidx_to_act[sampled_token_index]
        
        if sampled_activity == EOS: # end of sequence reached, i.e. event_to_predict == empty set
          #print(EOS)
          break
          
        # check if the sampled activity is actually a possible resolution for that trace
        # if not take the prediction with the 2nd highest prob... etc.
        while not sampled_activity in (events_to_predict):
          output_tokens[0, -1, sampled_token_index] = 0.0             # set the old idx with max prob to zero
          sampled_token_index = np.argmax(output_tokens[0, -1, :])
          sampled_activity = OUTidx_to_act[sampled_token_index]

        # the predicted activity must be removed from the activity to be predicted, so it won't be predicted again --> obtain possible resolution
        events_to_predict.remove(sampled_activity)

        decoded_trace.append(sampled_activity)

        #update the target sequence (len 1)
        target_seq = np.zeros((1, 1, n_dec_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        #update states
        states_value = [h, c]

    return decoded_trace

### dynamic evaluation

In [ ]:
# -> instead of decoding we stay in order and use the unencoded inputs 
for idx in range(0,20):
    enc_input_seq = test_encoder_input_data[idx:idx+1]
    decoded_trace = decode_seq1(enc_input_seq, dec_X_test[idx],
                                           OUTactivity_to_idx2,
                                           OUTidx_to_activity2)
    print('-'*20)
    print("Source   :" ,dec_X_test[idx])
    print("Predicted:",decoded_trace)
    print("Target   :", dec_y_test[idx])
    print()

In [ ]:
total = test_encoder_input_data.shape[0]
count = 0
for idx in range(total):
  enc_input_seq = test_encoder_input_data[idx:idx+1]
  decoded_trace = decode_seq1(enc_input_seq, dec_X_test[idx],
                                         OUTactivity_to_idx2,
                                         OUTidx_to_activity2)
  if dec_y_test[idx]  == decoded_trace[:]:
    count += 1
  print("Source   :" ,dec_X_test[idx])
  print("Predicted:",decoded_trace)
  print("Target   :", dec_y_test[idx])
  print()
print(count / total)

In [ ]:
# build funtion to decode predictions 
def decode_seq1(enc_input_seq, dec_input_seq, OUTact_to_idx, OUTidx_to_act):
    # encode the input sequence to get the internal state vectors
    states_value = encoder_model.predict(enc_input_seq)

    # generate empty target sequence of len 1 with only the start character
    target_seq = np.zeros((1,1, n_dec_tokens))
    target_seq[0, 0, OUTact_to_idx[BOS]] = 1.0

    #output seq loop
    stop_cond = False
    decoded_trace = []
    n_dec_events = 0
    n_repetitions = len(dec_input_seq[n_dec_events])
    while not stop_cond:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # sample token and add corresponding activity to the decoded trace
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_activity = [OUTidx_to_act[sampled_token_index]] 
        
        if len(dec_input_seq[n_dec_events]) > 1: # if we have an event set > 1 in input we need to consider it multiple times
            must_be_in = pos_res_for_unc_seq[tuple(sorted(dec_input_seq[n_dec_events]))] + [[activity] for activity in pos_res_for_unc_seq[tuple(sorted(dec_input_seq[n_dec_events]))][0]]
        else:
            must_be_in = pos_res_for_unc_seq[tuple(sorted(dec_input_seq[n_dec_events]))]
    
        #print("pred: ", sampled_activity)
        #print("must be in: ", must_be_in)
        
        # check if the sampled activity is actually a possible resolution for that trace
        while not sampled_activity in (must_be_in):
            # if not take the prediction with the 2nd highest prob... etc.
            output_tokens[0, -1, sampled_token_index] = 0.0 # set the old idx with max prob to zero
            sampled_token_index = np.argmax(output_tokens[0, -1, :])
            sampled_activity = [OUTidx_to_act[sampled_token_index]]
        
        decoded_trace.append(sampled_activity)
        
        if n_repetitions > 1:
            n_repetitions -= 1
        else:
            n_dec_events += 1
            if n_dec_events < len(dec_input_seq): 
                n_repetitions = len(dec_input_seq[n_dec_events])

        #check for stop condition: either hitting max length or prediciting EOS
        if (sampled_activity == tuple([EOS]) or len(decoded_trace) > max_dec_seq_len or
            n_dec_events >= len(dec_input_seq) ):
            stop_cond = True

        #update the target sequence (len 1)
        target_seq = np.zeros((1, 1, n_dec_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        #update states
        states_value = [h, c]

    return list(itertools.chain(*decoded_trace))